In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
from cv2 import imread

from trainer import Trainer
from img_dataset import ImgDataset
from classifier import ResNet, BasicBlock, ConvFCNetv2, ConvFCNetv3

### Finding device

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


### Defining Classifier

In [3]:
# Inherit from nn.Module
class ConvFCNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cnn = nn.Sequential(
            # Input size: 224 x 224 x 3
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1), # 224 x 224 x 16
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1), # 224 x 224 x 32
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2), # 112 x 112 x 32
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=2), # 56 x 56 x 64,
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1), # 56 x 56 x 64
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2), # 28 x 28 x 128,
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1), # 28 x 28 x 128
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2), # 14 x 14 x 128
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=2, padding=2), # 7 x 7 x 256
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, stride=2, padding=2), # 4 x 4 x 256
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=4, stride=4) # 1 x 1 x 256
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=150528, out_features=256),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            
            nn.Linear(in_features=128, out_features=32),
            nn.ReLU(),
            
            nn.Linear(in_features=32, out_features=1),
            # End with sigmoid function
            nn.Sigmoid()
        )
        
        self.flatten_image = nn.Flatten()
        
        self.flatten_cnn = nn.Flatten()
    
    # Forward is forward propergation
    def forward(self, x):
        out1 = self.cnn(x)
        
        flatten_x = self.flatten_image(x)
        out2 = self.fc(flatten_x)
        
        # out = out.view(out.size()[0], -1)
        flatten_out1 = self.flatten_cnn(out1)
        
        cat = torch.cat((flatten_out1, out2), dim=1)
        
        out = self.fc2(cat)
        return out

### Loading Testing Dataset

In [4]:
# Read training dataset
val_x = np.load("./data_chunk/test_x_chunk0.npy")
val_y = np.load("./data_chunk/test_y_chunk0.npy")[:, None]

print(type(val_x), val_x.shape)
print(type(val_y), val_y.shape)

<class 'numpy.ndarray'> (16703, 224, 224, 3)
<class 'numpy.ndarray'> (16703, 1)


### Loading Training Dataset (Chunk 0)

In [6]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk0.npy")
train_y = np.load("./data_chunk/train_y_chunk0.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (10000, 224, 224, 3)
<class 'numpy.ndarray'> (10000, 1)


In [7]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [8]:
# model = ResNet(img_channels=3, num_layers=18, num_classes=1, block=BasicBlock)
# model_chunk0 = ConvFCNet()
# model_chunk0 = torch.load("./results/model_convfcnet_epoch10_acc8733")
model_chunk0 = torch.load("./transplanted_model_acc9032")
num_epoch = 10
lr = 1e-5
optimizer = optim.Adam(model_chunk0.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk0, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 73.69 sec(s) Train Acc: 0.825200 Loss: 0.007865 | Val Acc: 0.856193 loss: 0.006095
[002/010] 73.29 sec(s) Train Acc: 0.870700 Loss: 0.005532 | Val Acc: 0.881937 loss: 0.004967
[003/010] 73.68 sec(s) Train Acc: 0.873400 Loss: 0.004936 | Val Acc: 0.822248 loss: 0.006123
[004/010] 74.10 sec(s) Train Acc: 0.877900 Loss: 0.004571 | Val Acc: 0.877687 loss: 0.004580
[005/010] 73.96 sec(s) Train Acc: 0.888400 Loss: 0.004209 | Val Acc: 0.868706 loss: 0.004837
[006/010] 74.03 sec(s) Train Acc: 0.894400 Loss: 0.004014 | Val Acc: 0.853739 loss: 0.005306
[007/010] 74.12 sec(s) Train Acc: 0.893000 Loss: 0.004003 | Val Acc: 0.871819 loss: 0.004693
[008/010] 74.10 sec(s) Train Acc: 0.900800 Loss: 0.003682 | Val Acc: 0.895468 loss: 0.004010
[009/010] 74.14 sec(s) Train Acc: 0.908600 Loss: 0.003513 | Val Acc: 0.891097 loss: 0.004103
[010/010] 74.38 sec(s) Train Acc: 0.913900 Loss: 0.003413 | Val Acc: 0.900856 loss: 0.003813
Val Acc: 0.900856 loss: 0.003813


In [9]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc9009")

### Loading Training Dataset (Chunk 1)

In [10]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk1.npy")
train_y = np.load("./data_chunk/train_y_chunk1.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (10000, 224, 224, 3)
<class 'numpy.ndarray'> (10000, 1)


In [11]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [12]:
model_chunk1 = torch.load("./results/model_convfcnetv3_epoch10_acc9009")
num_epoch = 10
lr = 1e-5
optimizer = optim.Adam(model_chunk1.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk1, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 73.58 sec(s) Train Acc: 0.891300 Loss: 0.004103 | Val Acc: 0.898222 loss: 0.003880
[002/010] 73.93 sec(s) Train Acc: 0.895700 Loss: 0.003920 | Val Acc: 0.900557 loss: 0.003740
[003/010] 74.23 sec(s) Train Acc: 0.897600 Loss: 0.003811 | Val Acc: 0.898282 loss: 0.003906
[004/010] 74.25 sec(s) Train Acc: 0.904400 Loss: 0.003570 | Val Acc: 0.867808 loss: 0.004965
[005/010] 74.49 sec(s) Train Acc: 0.904400 Loss: 0.003539 | Val Acc: 0.886068 loss: 0.004338
[006/010] 74.47 sec(s) Train Acc: 0.911900 Loss: 0.003377 | Val Acc: 0.899000 loss: 0.003814
[007/010] 74.61 sec(s) Train Acc: 0.913800 Loss: 0.003388 | Val Acc: 0.892774 loss: 0.004151
[008/010] 74.34 sec(s) Train Acc: 0.920200 Loss: 0.003198 | Val Acc: 0.903071 loss: 0.003701
[009/010] 74.55 sec(s) Train Acc: 0.914900 Loss: 0.003300 | Val Acc: 0.871161 loss: 0.004819
[010/010] 74.50 sec(s) Train Acc: 0.924000 Loss: 0.002995 | Val Acc: 0.901275 loss: 0.003734
Val Acc: 0.901275 loss: 0.003734


In [13]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc9013")

### Loading Training Dataset (Chunk 2)

In [14]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk2.npy")
train_y = np.load("./data_chunk/train_y_chunk2.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (10000, 224, 224, 3)
<class 'numpy.ndarray'> (10000, 1)


In [15]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [16]:
model_chunk2 = torch.load("./results/model_convfcnetv3_epoch10_acc9013")
num_epoch = 10
lr = 1e-6
optimizer = optim.Adam(model_chunk2.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk2, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 73.86 sec(s) Train Acc: 0.910500 Loss: 0.003369 | Val Acc: 0.914027 loss: 0.003327
[002/010] 74.34 sec(s) Train Acc: 0.913900 Loss: 0.003328 | Val Acc: 0.916901 loss: 0.003269
[003/010] 74.48 sec(s) Train Acc: 0.917200 Loss: 0.003079 | Val Acc: 0.916602 loss: 0.003276
[004/010] 74.68 sec(s) Train Acc: 0.915300 Loss: 0.003205 | Val Acc: 0.917620 loss: 0.003194
[005/010] 74.73 sec(s) Train Acc: 0.921000 Loss: 0.003046 | Val Acc: 0.917320 loss: 0.003233
[006/010] 74.73 sec(s) Train Acc: 0.920400 Loss: 0.003118 | Val Acc: 0.919715 loss: 0.003162
[007/010] 74.71 sec(s) Train Acc: 0.920700 Loss: 0.003027 | Val Acc: 0.920374 loss: 0.003121
[008/010] 74.67 sec(s) Train Acc: 0.919200 Loss: 0.003091 | Val Acc: 0.921691 loss: 0.003085
[009/010] 74.65 sec(s) Train Acc: 0.923000 Loss: 0.003006 | Val Acc: 0.921152 loss: 0.003080
[010/010] 75.11 sec(s) Train Acc: 0.918900 Loss: 0.003125 | Val Acc: 0.918937 loss: 0.003177
Val Acc: 0.918937 loss: 0.003177


In [17]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc9189")

### Loading Training Dataset (Chunk 3)

In [18]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk3.npy")
train_y = np.load("./data_chunk/train_y_chunk3.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (10000, 224, 224, 3)
<class 'numpy.ndarray'> (10000, 1)


In [19]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [20]:
model_chunk3 = torch.load("./results/model_convfcnetv3_epoch10_acc9189")
num_epoch = 10
lr = 1e-6
optimizer = optim.Adam(model_chunk3.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk3, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 73.67 sec(s) Train Acc: 0.926400 Loss: 0.002970 | Val Acc: 0.920374 loss: 0.003078
[002/010] 74.15 sec(s) Train Acc: 0.925900 Loss: 0.002918 | Val Acc: 0.920673 loss: 0.003048
[003/010] 74.34 sec(s) Train Acc: 0.925200 Loss: 0.002947 | Val Acc: 0.922409 loss: 0.002991
[004/010] 74.54 sec(s) Train Acc: 0.924000 Loss: 0.002897 | Val Acc: 0.923307 loss: 0.003011
[005/010] 74.68 sec(s) Train Acc: 0.926100 Loss: 0.002961 | Val Acc: 0.920314 loss: 0.003102
[006/010] 74.58 sec(s) Train Acc: 0.927300 Loss: 0.002885 | Val Acc: 0.920613 loss: 0.003033
[007/010] 74.64 sec(s) Train Acc: 0.924800 Loss: 0.002928 | Val Acc: 0.923367 loss: 0.002977
[008/010] 74.63 sec(s) Train Acc: 0.928900 Loss: 0.002809 | Val Acc: 0.921751 loss: 0.003045
[009/010] 74.54 sec(s) Train Acc: 0.929300 Loss: 0.002770 | Val Acc: 0.921691 loss: 0.003017
[010/010] 74.64 sec(s) Train Acc: 0.928300 Loss: 0.002847 | Val Acc: 0.922888 loss: 0.003030
Val Acc: 0.922888 loss: 0.003030


In [21]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc9229")

### Loading Training Dataset (Chunk 4)

In [5]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk4.npy")
train_y = np.load("./data_chunk/train_y_chunk4.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (10000, 224, 224, 3)
<class 'numpy.ndarray'> (10000, 1)


In [6]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [7]:
model_chunk4 = torch.load("./results/model_convfcnetv3_epoch10_acc9229")
num_epoch = 10
lr = 1e-6
optimizer = optim.Adam(model_chunk4.parameters(), lr=lr, weight_decay=4e-5)
# optimizer = optim.SGD(model_chunk4.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk4, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 74.32 sec(s) Train Acc: 0.920500 Loss: 0.003068 | Val Acc: 0.923008 loss: 0.002990
[002/010] 74.21 sec(s) Train Acc: 0.928800 Loss: 0.002915 | Val Acc: 0.923547 loss: 0.002985
[003/010] 74.34 sec(s) Train Acc: 0.919600 Loss: 0.003078 | Val Acc: 0.922289 loss: 0.002984
[004/010] 74.40 sec(s) Train Acc: 0.927200 Loss: 0.002880 | Val Acc: 0.924325 loss: 0.002963
[005/010] 74.49 sec(s) Train Acc: 0.923700 Loss: 0.002931 | Val Acc: 0.924804 loss: 0.002909
[006/010] 75.04 sec(s) Train Acc: 0.927200 Loss: 0.002840 | Val Acc: 0.923786 loss: 0.002977
[007/010] 74.63 sec(s) Train Acc: 0.928100 Loss: 0.002844 | Val Acc: 0.924564 loss: 0.002965
[008/010] 74.65 sec(s) Train Acc: 0.925900 Loss: 0.002886 | Val Acc: 0.924624 loss: 0.002944
[009/010] 74.59 sec(s) Train Acc: 0.927600 Loss: 0.002739 | Val Acc: 0.924744 loss: 0.002931
[010/010] 74.63 sec(s) Train Acc: 0.928900 Loss: 0.002812 | Val Acc: 0.924744 loss: 0.002931
Val Acc: 0.924744 loss: 0.002931


In [8]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc9247")

### Loading Training Dataset (Chunk 5)

In [5]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk5.npy")
train_y = np.load("./data_chunk/train_y_chunk5.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (10000, 224, 224, 3)
<class 'numpy.ndarray'> (10000, 1)


In [6]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [8]:
model_chunk5 = torch.load("./results/model_convfcnetv3_epoch10_acc9247")
num_epoch = 10
lr = 1e-7
optimizer = optim.Adam(model_chunk5.parameters(), lr=lr)#, weight_decay=4e-5)
# optimizer = optim.SGD(model_chunk5.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk5, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 73.73 sec(s) Train Acc: 0.919600 Loss: 0.003115 | Val Acc: 0.924385 loss: 0.002901
[002/010] 73.74 sec(s) Train Acc: 0.919900 Loss: 0.003067 | Val Acc: 0.924325 loss: 0.002920
[003/010] 74.04 sec(s) Train Acc: 0.919500 Loss: 0.003049 | Val Acc: 0.924145 loss: 0.002928
[004/010] 74.13 sec(s) Train Acc: 0.919900 Loss: 0.003016 | Val Acc: 0.924445 loss: 0.002887
[005/010] 74.17 sec(s) Train Acc: 0.920600 Loss: 0.003054 | Val Acc: 0.924205 loss: 0.002925
[006/010] 74.38 sec(s) Train Acc: 0.919800 Loss: 0.003030 | Val Acc: 0.920553 loss: 0.003032
[007/010] 74.30 sec(s) Train Acc: 0.920000 Loss: 0.003027 | Val Acc: 0.925462 loss: 0.002910
[008/010] 74.29 sec(s) Train Acc: 0.924800 Loss: 0.002929 | Val Acc: 0.924744 loss: 0.002907
[009/010] 74.38 sec(s) Train Acc: 0.921000 Loss: 0.003040 | Val Acc: 0.923846 loss: 0.002935
[010/010] 74.36 sec(s) Train Acc: 0.921900 Loss: 0.002967 | Val Acc: 0.927079 loss: 0.002858
Val Acc: 0.927079 loss: 0.002858


In [9]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc9271")

### Loading Training Dataset (Chunk 6)

In [10]:
# Read training dataset
train_x = np.load("./data_chunk/train_x_chunk6.npy")
train_y = np.load("./data_chunk/train_y_chunk6.npy")[:, None]

print(type(train_x), train_x.shape)
print(type(train_y), train_y.shape)

<class 'numpy.ndarray'> (6811, 224, 224, 3)
<class 'numpy.ndarray'> (6811, 1)


In [11]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    # Add any data argumentation here
    # e.g:
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(15),
    # transforms.RandomResizedCrop(),
    # transforms.Normalize(mean, std)
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Wrap the x and y using ImgDataset
train_dataset = ImgDataset(train_x, train_y, train_transforms)
val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [12]:
model_chunk6 = torch.load("./results/model_convfcnetv3_epoch10_acc9271")
num_epoch = 10
lr = 1e-7
optimizer = optim.Adam(model_chunk6.parameters(), lr=lr, weight_decay=4e-5)
loss = nn.BCELoss()
batch_size = 64

trainer = Trainer(model_chunk6, optimizer, num_epoch, loss, train_dataset, val_dataset, batch_size, device)

trainer.train()
trainer.validation()

[001/010] 58.23 sec(s) Train Acc: 0.917486 Loss: 0.003189 | Val Acc: 0.926241 loss: 0.002915
[002/010] 58.38 sec(s) Train Acc: 0.921597 Loss: 0.003100 | Val Acc: 0.926540 loss: 0.002861
[003/010] 58.49 sec(s) Train Acc: 0.919542 Loss: 0.003098 | Val Acc: 0.925822 loss: 0.002922
[004/010] 58.58 sec(s) Train Acc: 0.924240 Loss: 0.002970 | Val Acc: 0.927917 loss: 0.002866
[005/010] 58.83 sec(s) Train Acc: 0.921304 Loss: 0.003045 | Val Acc: 0.926839 loss: 0.002852
[006/010] 58.92 sec(s) Train Acc: 0.926589 Loss: 0.002975 | Val Acc: 0.925223 loss: 0.002921
[007/010] 58.87 sec(s) Train Acc: 0.924681 Loss: 0.002956 | Val Acc: 0.927738 loss: 0.002853
[008/010] 58.69 sec(s) Train Acc: 0.924093 Loss: 0.003013 | Val Acc: 0.928216 loss: 0.002849
[009/010] 58.75 sec(s) Train Acc: 0.918661 Loss: 0.003160 | Val Acc: 0.926540 loss: 0.002871
[010/010] 58.75 sec(s) Train Acc: 0.920570 Loss: 0.003066 | Val Acc: 0.926361 loss: 0.002858
Val Acc: 0.926361 loss: 0.002858


In [ ]:
torch.save(trainer.model, "./results/model_convfcnetv3_epoch10_acc")